In [3]:
import logging
import time
import os
import yaml
from datetime import datetime

import enoslib as en
import g5k
import pandas as pd


en.init_logging(level=logging.INFO)

en.check()

_____        ___  ____  _ _ _
 | ____|_ __  / _ \/ ___|| (_) |__
 |  _| | '_ \| | | \___ \| | | '_ \
 | |___| | | | |_| |___) | | | |_) |
 |_____|_| |_|\___/|____/|_|_|_.__/  10.2.0

 • Documentation: ]8;id=901468;https://discovery.gitlabpages.inria.fr/enoslib/\https://discovery.gitlabpages.inria.fr/enoslib/]8;;\                            
 • Source: ]8;id=131544;https://gitlab.inria.fr/discovery/enoslib\https://gitlab.inria.fr/discovery/enoslib]8;;\                                         
 • Chat: ]8;id=764534;https://framateam.org/enoslib\https://framateam.org/enoslib]8;;\

                         Dependency check                         
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Provider      ┃    Status     ┃ Hint                           ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Chameleon     │ NOT INSTALLED │ pip install enoslib[chameleon] │
│ ChameleonKVM  │ NOT INSTALLED │ pip install enoslib[chameleon] │
│ ChameleonEdge │ NOT INSTALLED │ pip install enoslib[chameleon] │
│ Distem        │ NOT INSTALLED │ pip install enoslib[distem]    │
│ IOT-lab       │ NOT INSTALLED │ pip install enoslib[iotlab]    │
│ Grid'5000     │   INSTALLED   │                                │
│ Openstack     │ NOT INSTALLED │ pip install enoslib[chameleon] │
│ Vagrant       │ NOT INSTALLED │ pip install enoslib[vagrant]   │
│ VMonG5k       │   INSTALLED   │                                │
└───────────────┴───────────────┴────────────────────────────────┘

                                Connectivity check                                 
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Provider  ┃ Key                 ┃ Connectivity ┃ Hint                           ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Grid'5000 │ ssh:access          │      ✅      │ Connection to access.grid5000… │
│ Grid'5000 │ ssh:access:frontend │      ✅      │ Connection Host(rennes.grid50… │
│ Grid'5000 │ api:access          │      ✅      │                                │
│ VMonG5k   │ access              │      ❔      │ Check G5k status               │
└───────────┴─────────────────────┴──────────────┴────────────────────────────────┘

In [4]:
import subprocess

def collect_results(path_locust, path_wattmetre, path_result, roles, planned_monitor_end, planned_monitor_start):
    monitor_server = en.Kwollect(nodes=roles["server"])
    monitor_server.deploy()
    monitor_server.start(planned_monitor_start)
    monitor_server.stop(planned_monitor_end)
    wattmetre_metrics = monitor_server.get_metrics_pandas(metrics=["wattmetre_power_watt"])
    wattmetre_metrics.to_csv(path_wattmetre)

    if os.path.isfile(path_locust+"locust_stats_history.csv"):
        print("Data already present")
    else:
        site = roles["stress"][0].address.split('.')[1]
        cmd = f"scp -r {site}.g5k:~/locust".split(" ")
        cmd.append(path_result)
        subprocess.run(cmd)

        if os.path.isfile(path_locust+"locust_stats_history.csv"):
            cmd = f"ssh {site}.g5k 'rm -r ~/locust/*.csv'"
            os.system(cmd)
        else:
            print("ERROR: Copy failed")

In [ ]:
conf = (
    en.G5kConf.from_settings(
        job_type=["monitor=wattmetre_power_watt"],
        job_name="seektune"
    ).add_machine(
        roles=["server"], servers=["XXX"] # Any G5K Machine, eg. chirop-1.lille.grid5000.fr 
    )
    .add_machine(
        roles=["stress"], servers=["XXX"] # Any G5K Machine, eg. chirop-1.lille.grid5000.fr 
    )
    .finalize()
)

provider = en.G5k(conf)

(roles, st_server_adress, st_node, env_line, site, cors, proxy) = g5k.reserve_resources(provider)

print(st_server_adress)

g5k.deploy_st("saas", roles, en, site, cors, env_line, prefork=True)

with en.actions(roles=roles["stress"]) as a:
    site = roles["stress"][0].address.split('.')[1]
    # Setup SSH
    a.copy(src="../g5kuserkey.pub", dest="~/.ssh/", mode="600")
    a.copy(src="../g5kuserkey", dest="~/.ssh/", mode="600")
    a.copy(src="../config", dest="~/.ssh/")

    # Clone Git Repo
    a.shell(chdir="/tmp", cmd=f"git clone git@YOUR_GIT_STRESS.git")

    # Configure KWollector
    a.lineinfile(path="$HOME/.profile", line=f"export G5K_SITE={site}")
    a.lineinfile(path="$HOME/.bashrc", line=f"export G5K_SITE={site}")

    # Install Locust
    a.shell(cmd="pip install locust")

    a.shell(cmd="tmux new-session -d -s stress")

    results = a.results

print([res.stdout for res in results.filter(task="shell")])

stress_config = {
    'node':st_node,
    'time': -1,
    'mode': 'saas', 
    'cstate': False,
    'args': {
        'spawn-rate':1,
        'run-time':"1h",
        'users': 1000,
        'host': f'http://{st_server_adress}'
    }
}

def prepare_server_for_stresstest(cstate):
    print("Preparing server for stress...")
    if cstate:
        en.run_command("cpupower idle-set -E", roles=roles["server"])
    else:
        en.run_command("cpupower idle-set -D 0", roles=roles["server"])
    
    time.sleep(5)
    print("Server prepared.")

def createArgs(conf):
    builder = " "
    for k,v in conf.items():
        builder = builder + f"--{k} {v} "
    
    return builder

prepare_server_for_stresstest(stress_config["cstate"])

ts = datetime.now()
stress_config["time"] = ts
path_result = f"../RESULTS_G5K/{ts}"
path_config = f"{path_result}/config.yml"
path_throughput = f"{path_result}/throughput.csv"
path_wattmetre = f"{path_result}/wattmetre.csv"
path_locust = f'{path_result}/locust/'

os.mkdir(path_result)
os.mkdir(path_locust)
with open(path_config, "a") as conf:
    yaml.dump(stress_config, conf, default_flow_style=False)

monitor = en.Kwollect(nodes=roles["server"])
monitor.deploy()
monitor.start()
planned_monitor_start = time.time()

# Reference measure
time.sleep(300)
planned_monitor_end = time.time()+pd.Timedelta(stress_config['args']['run-time']).total_seconds()

print(f'planned_monitor_start= {planned_monitor_start}')
print(f'planned_monitor_end= {planned_monitor_end}')

with en.actions(roles=roles["stress"]) as a:
    args = createArgs(stress_config["args"])

    a.shell(chdir="/tmp", cmd="rm -f stresstest.lock")

    print("Starting stress test with args:")
    print(args)
    # Start stress tester
    a.shell(cmd=f"tmux send-keys -t stress.0 \"cd /tmp/seek-tune-stress\" ENTER \"locust -f {stress_config["mode"]}.py {args} --headless --csv /home/YOUR HOME/locust/locust --csv-full-history --processes -1 && touch /tmp/stresstest.lock\" ENTER")
    # a.wait_for(path="/tmp/stresstest.lock")

time.sleep(planned_monitor_end - time.time())

exists = False
while not exists:
    print("Waiting")
    time.sleep(30)
    host = ".".join(roles["stress"][0].address.split(".")[0:2]) + ".g5k"
    cmd = f'ssh {host} "test -e /tmp/stresstest.lock"'
    res = os.system(cmd)
    exists = res == 0

collect_results(path_locust, path_wattmetre, path_result, roles, planned_monitor_end, planned_monitor_start)

### In case the script crashed while waiting for the experiment to finish

In [28]:
collect_results(path_locust, path_wattmetre, path_result, roles, planned_monitor_end, planned_monitor_start)